In [ ]:
! pip install parsivar

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 36.2 MB 425 kB/s 
     |████████████████████████████████| 1.5 MB 37.0 MB/s 
  Created wheel for parsivar: filename=parsivar-0.2.3-py3-none-any.whl size=36492972 sha256=3d80483dead92803a6a2a742ed174966163e579bfea6800288c9f9de80ebd9c7
  Stored in directory: /root/.cache/pip/wheels/ae/67/7a/49cbf08f64d3f76a26eceaf0e481a40e233f05d4356875cbed
  Created wheel for nltk: filename=nltk-3.4.5-py3-none-any.whl size=1449922 sha256=3211cad70981bb9b028431a488f7bea4c26d9524b5d0c8c719e3350501db9d63
  Stored in directory: /root/.cache/pip/wheels/48/8b/7f/473521e0c731c6566d631b281f323842bbda9bd819eb9a3ead
Successfully built parsivar nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
! gdown 1Bo_WQLGQD5UrPv3IJdV7cdVZtSHeJSQB
! unzip data.zip

Downloading...
From: https://drive.google.com/uc?id=1Bo_WQLGQD5UrPv3IJdV7cdVZtSHeJSQB
To: /content/data.zip
100% 2.45M/2.45M [00:00<00:00, 184MB/s]
Archive:  data.zip
   creating: data/
  inflating: data/test.en            
  inflating: data/test.fa            
  inflating: data/train.en           
  inflating: data/train.fa           
  inflating: data/valid.en           
  inflating: data/valid.fa           


## Preprocess

In [ ]:
from parsivar import Normalizer
from tqdm import tqdm
!mkdir preprocessed_data
my_normalizer = Normalizer(pinglish_conversion_needed = True,
                           statistical_space_correction = True)
files = ['test.en','test.fa','train.en','train.fa','valid.en','valid.fa']
for file_name in tqdm(files):
  with open('data/'+file_name) as f:
    lines = f.readlines()
    lines = [line.rstrip() for line in lines]
  for line in lines:
    if(file_name.endswith(".fa")):
      text = my_normalizer.normalize(line)
    else:
      text = line.lower()
    with open('preprocessed_data/'+file_name, 'a') as f:
      f.write(text+'\n')

100%|██████████| 6/6 [00:32<00:00,  5.45s/it]


In [ ]:
! zip -r preprocessed_data.zip preprocessed_data

  adding: preprocessed_data/ (stored 0%)
  adding: preprocessed_data/train.fa (deflated 69%)
  adding: preprocessed_data/test.en (deflated 62%)
  adding: preprocessed_data/valid.en (deflated 56%)
  adding: preprocessed_data/train.en (deflated 59%)
  adding: preprocessed_data/test.fa (deflated 72%)
  adding: preprocessed_data/valid.fa (deflated 68%)


## OpenNMT

### BPE

In [ ]:
!git clone -b legacy https://github.com/OpenNMT/OpenNMT-py

Cloning into 'OpenNMT-py'...
remote: Enumerating objects: 17675, done.
remote: Total 17675 (delta 0), reused 0 (delta 0), pack-reused 17675
Receiving objects: 100% (17675/17675), 273.61 MiB | 38.30 MiB/s, done.
Resolving deltas: 100% (12760/12760), done.


In [ ]:
!pip install -r ./OpenNMT-py/floyd_requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/pytorch/text to /tmp/pip-req-build-7hwlxzop
  Running command git clone -q https://github.com/pytorch/text /tmp/pip-req-build-7hwlxzop
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  ERROR: Failed building wheel for torchtext
Failed to build torchtext
ERROR: Could not build wheels for torchtext which use PEP 517 and cannot be installed directly


In [ ]:
! mkdir BPE

In [ ]:
!python OpenNMT-py/tools/learn_bpe.py -i preprocessed_data/train.en -o BPE/src.code -s 10000

In [ ]:
!python OpenNMT-py/tools/learn_bpe.py -i preprocessed_data/train.fa -o BPE/tgt.code -s 10000

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c BPE/src.code -i preprocessed_data/train.en -o BPE/train-bpe.en

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c BPE/src.code -i preprocessed_data/valid.en -o BPE/valid-bpe.en

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c BPE/src.code -i preprocessed_data/test.en -o BPE/test-bpe.en.txt

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c BPE/tgt.code -i preprocessed_data/train.fa -o BPE/train-bpe.fa

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c BPE/tgt.code -i preprocessed_data/valid.fa -o BPE/valid-bpe.fa

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c BPE/tgt.code -i preprocessed_data/test.fa -o BPE/valid-bpe.fa

In [ ]:
!nvidia-smi

Mon Jun  6 11:59:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### openNMT preprocess

In [ ]:
!pip install torchtext==0.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.9 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.12.0
    Uninstalling torchtext-0.12.0:
      Successfully uninstalled torchtext-0.12.0


In [ ]:
!pip install configargparse

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!mkdir translate

In [ ]:
!python OpenNMT-py/preprocess.py -train_src BPE/train-bpe.en -train_tgt BPE/train-bpe.fa -valid_src BPE/valid-bpe.en -valid_tgt BPE/valid-bpe.fa -save_data translate/data -overwrite

[2022-06-06 14:48:47,714 INFO] Extracting features...
[2022-06-06 14:48:47,714 INFO]  * number of source features: 0.
[2022-06-06 14:48:47,714 INFO]  * number of target features: 0.
[2022-06-06 14:48:47,714 INFO] Building `Fields` object...
[2022-06-06 14:48:47,714 INFO] Building & saving training data...
[2022-06-06 14:48:47,715 WARNING] Shards for corpus train already exist, will be overwritten because `-overwrite` option is set.
[2022-06-06 14:48:47,721 WARNING] Overwrite shards for corpus None
[2022-06-06 14:48:47,839 INFO] Building shard 0.
[2022-06-06 14:48:50,145 INFO]  * saving 0th train data shard to translate/data.train.0.pt.
[2022-06-06 14:48:52,362 INFO]  * tgt vocab size: 10073.
[2022-06-06 14:48:52,372 INFO]  * src vocab size: 9808.
[2022-06-06 14:48:52,424 INFO] Building & saving validation data...
[2022-06-06 14:48:52,424 WARNING] Shards for corpus valid already exist, will be overwritten because `-overwrite` option is set.
[2022-06-06 14:48:52,430 WARNING] Overwrite sh

### train

In [ ]:
!python OpenNMT-py/train.py -data translate/data -save_model translate/model -train_steps 50000 -save_checkpoint_steps 5000 -valid_steps 1000 -world_size 1 -gpu_rank 0

[2022-06-06 14:49:23,076 INFO]  * src vocab size = 9808
[2022-06-06 14:49:23,076 INFO]  * tgt vocab size = 10073
[2022-06-06 14:49:23,076 INFO] Building model...
[2022-06-06 14:49:27,189 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(9808, 500, padding_idx=1)
        )
      )
    )
    (rnn): LSTM(500, 500, num_layers=2, dropout=0.3)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(10073, 500, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.3, inplace=False)
    (rnn): StackedLSTM(
      (dropout): Dropout(p=0.3, inplace=False)
      (layers): ModuleList(
        (0): LSTMCell(1000, 500)
        (1): LSTMCell(500, 500)
      )
    )
    (attn): GlobalAttention(
      (linear_in): Linear(in_features=500, out_features=500, bias=False)
      (

### test

In [ ]:
!mkdir pred
!mkdir pred/test
!mkdir pred/validation

In [ ]:
bleus = []
for i in tqdm(range(5000,50001,5000)) :
  c1 = !python OpenNMT-py/translate.py -model translate/model_step_{i}.pt -src data/valid.en -output pred/validation/pred_valid_{i}.txt -replace_unk
  c2 = !sed -i "s/@@ //g"  pred/validation/pred_valid_{i}.txt
  c3 = !perl OpenNMT-py/tools/multi-bleu.perl data/valid.fa < pred/validation/pred_valid_{i}.txt
  bleus.append(c3)

100%|██████████| 10/10 [04:04<00:00, 24.41s/it]


In [ ]:
bleus

[['BLEU = 2.04, 20.3/3.5/0.9/0.3 (BP=1.000, ratio=1.163, hyp_len=7351, ref_len=6322)'],
 ['BLEU = 3.14, 29.8/6.8/2.0/0.5 (BP=0.850, ratio=0.860, hyp_len=5437, ref_len=6322)'],
 ['BLEU = 3.84, 32.2/7.6/2.3/0.8 (BP=0.842, ratio=0.853, hyp_len=5394, ref_len=6322)'],
 ['BLEU = 3.86, 32.3/7.9/2.4/0.8 (BP=0.808, ratio=0.824, hyp_len=5210, ref_len=6322)'],
 ['BLEU = 4.44, 31.1/7.8/2.6/0.9 (BP=0.907, ratio=0.911, hyp_len=5760, ref_len=6322)'],
 ['BLEU = 4.32, 31.6/7.9/2.5/0.9 (BP=0.885, ratio=0.891, hyp_len=5633, ref_len=6322)'],
 ['BLEU = 4.04, 31.0/7.2/2.3/0.9 (BP=0.862, ratio=0.870, hyp_len=5502, ref_len=6322)'],
 ['BLEU = 3.96, 31.9/7.7/2.4/0.8 (BP=0.846, ratio=0.857, hyp_len=5416, ref_len=6322)'],
 ['BLEU = 4.07, 32.8/7.9/2.5/0.9 (BP=0.842, ratio=0.853, hyp_len=5394, ref_len=6322)'],
 ['BLEU = 4.20, 30.5/7.5/2.5/0.8 (BP=0.908, ratio=0.912, hyp_len=5764, ref_len=6322)']]

In [ ]:
!python OpenNMT-py/translate.py -model translate/model_step_50000.pt -src data/test.en -output pred/test/pred_test.txt -replace_unk -verbose

[2022-06-06 15:50:56,632 INFO] Translating shard 0.
/content/OpenNMT-py/onmt/translate/beam_search.py:209: UserWarning: An output with one or more elements was resized since it had shape [150], which does not match the required output shape [30, 5].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:24.)
  torch.mul(self.topk_scores, length_penalty, out=self.topk_log_probs)
/content/OpenNMT-py/onmt/translate/beam_search.py:212: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use 

In [ ]:
!sed -i "s/@@ //g"  pred/test/pred_test.txt

In [ ]:
!perl OpenNMT-py/tools/multi-bleu.perl data/test.fa < pred/test/pred_test.txt

BLEU = 3.94, 33.7/8.0/2.3/0.6 (BP=0.891, ratio=0.897, hyp_len=10208, ref_len=11385)
